# Deploy DxSuite Fast OCR API Package from AWS Marketplace 


## Overview of DxSuite Fast OCR API
It is an OCR model targeted at DxSuite's standard forms.

This sample notebook shows you how to deploy [DxSuite Fast OCR API](https://aws.amazon.com/marketplace/pp/prodview-mdvbzhue5fpsi) using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [DxSuite Fast OCR API](https://aws.amazon.com/marketplace/pp/prodview-mdvbzhue5fpsi). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Clean-up](#3.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [DxSuite Fast OCR API](https://aws.amazon.com/marketplace/pp/prodview-mdvbzhue5fpsi).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
model_package_arn = "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/dxs-10-typical-ai-function-pac-c7b8a3f4cef935e1a6e3920cbbd3c14a"

In [ ]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import numpy as np
import struct
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
import japanize_matplotlib


In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
model_name = "dx-suite-ocr-model"

content_type = 'application/octet-stream'

real_time_inference_instance_type = (
    'ml.g5.2xlarge'
)
batch_transform_inference_instance_type = (
    'ml.g5.2xlarge'
)

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [ ]:
workflow_id = '723cf4ee-8503-43e2-8def-b0c11d53bdf0'    # ID of the workflow you got template_info from.

In [ ]:
with open('data/input/real-time/template_info.zip', 'rb') as zip_file:
    zip_bytes = zip_file.read()
    
    # encode request_type
    request_type_bytes = struct.pack('I', 2)

    # Encode the size of the zip in binary format (4-byte integer)
    zip_size = struct.pack('I', len(zip_bytes))

    # Combine data
    register_data = (
            request_type_bytes +
            zip_size + zip_bytes
    )

In [ ]:
with open('data/input/real-time/image.png', 'rb') as image:
    image_bytes = image.read()
    
    # encode request_type
    request_type_bytes = struct.pack('I', 0)

    # encode workflow_id
    workflow_id_bytes = workflow_id.encode('utf-8')
    workflow_id_size = struct.pack('I', len(workflow_id_bytes))

    # Encode the size of the image in binary format (4-byte integer)
    image_size = struct.pack('I', len(image_bytes))

    # Encode do_align as bool type (1 byte)
    do_align = True
    do_align_byte = struct.pack('?', do_align)

    # Combine data
    predict_data = (
        request_type_bytes +
        workflow_id_size + workflow_id_bytes +
        image_size + image_bytes +
        do_align_byte
    )

In [ ]:
register_file_name = 'data/input/real-time/register_data'
register_output_file_name = 'data/output/real-time/register_output.json'
with open(register_file_name, 'wb') as f:
    f.write(register_data)

predict_file_name = 'data/input/real-time/predict_data'
predict_output_file_name = 'data/output/real-time/predict_output.json'
with open(predict_file_name, 'wb') as f:
    f.write(predict_data)

### C. Perform real-time inference

#### 1. Register the template

The first step is to register the template. This step is required only once for a template. The response will contain the template_id which you can use in the next step.

In [ ]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$register_file_name --content-type $content_type --region $sagemaker_session.boto_region_name $register_output_file_name

#### 2. Perform inference

In [ ]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$predict_file_name --content-type $content_type --region $sagemaker_session.boto_region_name $predict_output_file_name

### D. Visualize output

In [ ]:
with open(predict_output_file_name, 'r') as f:
    output = json.load(f)
    image = ImageEdit.open('data/input/real-time/image.png').convert('RGB')
    fig, ax = plt.subplots()
    ax.imshow(image)
    for data_item in output['data_items']:
        rect = patches.Rectangle((data_item['x'], data_item['y']), data_item['width'], data_item['height'], linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(data_item['x'], data_item['y'], data_item['result'], fontsize=12, color='r')
    plt.show()

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

